# Data collection

In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm

In [70]:
pd.set_option('display.max_columns', 50)


## Get all products via API  

In [7]:
BASE_URL = "http://namperfume.net/products"

In [64]:
products = []
for i in tqdm(range(1,56)):
    page=i
    page_size=60
    api_url = "https://api.namefragrance.vn/v2/products/public?store_id=60026"
    
    full_page_url = f"{api_url}&page={page}&limit={page_size}"
    response = requests.get(full_page_url, headers=headers)
    data = response.json()['items']  # Nếu là JSON
    if data:
        for p in data:
            item  = {}
            item['id'] = p['id']
            item['name'] = p['name']
            item['full_name'] = p['full_name']
            item['parent_handle'] = p['parent_handle']
            item['sku'] = p['sku']
            item['production_year'] = p['production_year']
            item['origin'] = p['origin']
            item['price'] = p['price']
            item['compare_at_price'] = p['compare_at_price']
            item['parent_id'] = p['parent_id']
            item['total_quantity'] = p['total_quantity']
            item['sold_quantity'] = p['sold_quantity']
            item['brand'] = p['brand']['title'] if p['brand'] else None
            item['product_gender'] = [i['value'] for i in p['tags'] if i['type'] == 'PRODUCT GENDER'] if p['tags'] else None
            item['product_style'] = [i['value'] for i in p['tags'] if i['name'] == 'Phong cách'] if p['tags'] else None
            item['product_note'] = [i['value'] for i in p['tags'] if i['name'] == 'Nhóm hương'] if p['tags'] else None
            item['rate'] = p['rate']['rate']
            item['count_rate'] = p['rate']['count_rate']
            item['count_rate_1'] = p['rate']['count_rate_1']
            item['count_rate_2'] = p['rate']['count_rate_2']
            item['count_rate_3'] = p['rate']['count_rate_3']
            item['count_rate_4'] = p['rate']['count_rate_4']
            item['count_rate_5'] = p['rate']['count_rate_5']
            
            products.append(item)
    else: 
        print(response)
        break

100%|██████████| 55/55 [00:06<00:00,  8.16it/s]


In [65]:
len(products)

3300

In [68]:
product_df = pd.DataFrame(products)

In [79]:
product_df

,id,name,full_name,parent_handle,sku,production_year,origin,price,compare_at_price,parent_id,total_quantity,sold_quantity,brand,product_gender,product_style,product_note,rate,count_rate,count_rate_1,count_rate_2,count_rate_3,count_rate_4,count_rate_5
0,1003772321,Dsquared2 Icon Pour Homme,Dsquared2 Icon Pour Homme - 100ml Tester,dsquared2-icon-pour-homme,110100204700,2024,Ý,2200000,0,1003772317,0,0,DSQUARED2,[Nam],"[Tinh tế, Nam tính, Cuốn hút]","[Hương gừng, Cây xô thơm]",0.0,0,0,0,0,0,0
1,1003772317,Dsquared2 Icon Pour Homme,Dsquared2 Icon Pour Homme - 100ml,dsquared2-icon-pour-homme,110100204699,2024,Ý,2200000,2211000,0,38,1,DSQUARED2,[Nam],"[Tinh tế, Nam tính, Cuốn hút]","[Hương gừng, Hoa Tulip, Cây xô thơm]",0.0,0,0,0,0,0,0
2,1003772262,Dsquared2 Icon Pour Femme,Dsquared2 Icon Pour Femme - 100ml Tester,dsquared2-icon-pour-femme,110100204698,2024,Ý,2200000,0,1003772251,0,0,DSQUARED2,[Nữ],"[Tươi mát, Tinh tế, Trẻ trung]","[Hương Cam quýt, Hoa nhài, Kẹo praline]",0.0,0,0,0,0,0,0
3,1003772251,Dsquared2 Icon Pour Femme,Dsquared2 Icon Pour Femme - 100ml,dsquared2-icon-pour-femme,110100204697,2024,Ý,2200000,2211000,0,39,2,DSQUARED2,[Nữ],"[Tươi mát, Tinh tế, Trẻ trung]","[Hương Cam quýt, Hoa nhài, Kẹo praline]",0.0,0,0,0,0,0,0
4,1003771959,Nishane Hacivat,Nishane Hacivat - 100ml Tester,nishane-hacivat-1,110100204696,2017,Thổ Nhĩ Kỳ,7850000,0,1003771949,0,0,Nishane,[Unisex],"[Tinh tế, Trưởng Thành, Cuốn hút]","[Hương rêu sồi, Quả dứa, Bưởi]",0.0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,10950126,Jo Malone London Mimosa & Cardamom,Jo Malone London Mimosa & Cardamom - 100ml,nuoc-hoa-unisex-jo-malone-mimosa-cardamom,110100200330,2015,Anh,2950000,3500000,0,0,0,Jo Malone,[Nữ],"[Thanh lịch, Sang trọng, Dịu dàng]","[Hoa mimosa, Bạch đậu khấu, Đậu tonka]",5.0,1,0,0,0,0,100
3296,10950125,Jo Malone London Grapefruit,Jo Malone London Grapefruit - 30ml,nuoc-hoa-unisex-jo-malone-grapefruit,110100200328,1992,Anh,1950000,2400000,0,0,0,Jo Malone,[Unisex],"[Tươi mới, Sang trọng, Tinh tế]","[Quả bưởi, Hoa hương thảo, Cỏ hương bài, Bạc hà]",4.5,2,0,0,0,50,50
3297,10950123,Jo Malone London Gardenia & Oud Absolu,Jo Malone London Gardenia & Oud Absolu - 100ml,jo-malone-london-gardenia-oud-absolu,110100202113,2020,Anh,4350000,4350000,0,0,0,Jo Malone,[Unisex],"[Tinh tế, Sang trọng, Thu Hút]","[Trầm hương, Hoa dành dành]",4.0,1,0,0,0,100,0
3298,10950121,Jo Malone London French Lime Blossom,Jo Malone London French Lime Blossom - 30ml,nuoc-hoa-nu-jo-malone-french-lime-blossom,110100200326,1995,Anh,1950000,2400000,0,0,0,Jo Malone,[Uni],"[Tươi mát, Nhẹ nhàng, Gợi cảm]","[Sáp tổ ong, Cam bergamot, Hoa linh lan, Hoa q...",5.0,1,0,0,0,0,100


## Get description  

In [91]:
l_perent_handle = product_df['parent_handle'].drop_duplicates().tolist()

In [96]:
des_dict = {}
for i in tqdm(l_perent_handle):
    response = requests.get(f"https://namperfume.net/products/{i}", headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    description = soup.find("div", class_="description-productdetail")

    if description:
        des_dict[i] = description.get_text(separator=' ',strip=True)  # Get clean text
    else:
        des_dict[i] = ''
    time.sleep(1)


100%|██████████| 1818/1818 [45:42<00:00,  1.51s/it] 


In [104]:
des_df = pd.DataFrame.from_dict(data=des_dict, orient='index').reset_index()\
.rename(columns={'index':'parent_handle', 0:'description'})

In [106]:
full_df = pd.merge(product_df, des_df, on='parent_handle', how='left')
full_df

,id,name,full_name,parent_handle,sku,production_year,origin,price,compare_at_price,parent_id,total_quantity,sold_quantity,brand,product_gender,product_style,product_note,rate,count_rate,count_rate_1,count_rate_2,count_rate_3,count_rate_4,count_rate_5,description
0,1003772321,Dsquared2 Icon Pour Homme,Dsquared2 Icon Pour Homme - 100ml Tester,dsquared2-icon-pour-homme,110100204700,2024,Ý,2200000,0,1003772317,0,0,DSQUARED2,[Nam],"[Tinh tế, Nam tính, Cuốn hút]","[Hương gừng, Cây xô thơm]",0.0,0,0,0,0,0,0,"Hương đầu: Gừng, Cam chanh Hương giữa: Hoa tul..."
1,1003772317,Dsquared2 Icon Pour Homme,Dsquared2 Icon Pour Homme - 100ml,dsquared2-icon-pour-homme,110100204699,2024,Ý,2200000,2211000,0,38,1,DSQUARED2,[Nam],"[Tinh tế, Nam tính, Cuốn hút]","[Hương gừng, Hoa Tulip, Cây xô thơm]",0.0,0,0,0,0,0,0,"Hương đầu: Gừng, Cam chanh Hương giữa: Hoa tul..."
2,1003772262,Dsquared2 Icon Pour Femme,Dsquared2 Icon Pour Femme - 100ml Tester,dsquared2-icon-pour-femme,110100204698,2024,Ý,2200000,0,1003772251,0,0,DSQUARED2,[Nữ],"[Tươi mát, Tinh tế, Trẻ trung]","[Hương Cam quýt, Hoa nhài, Kẹo praline]",0.0,0,0,0,0,0,0,"Hương đầu: Cam chanh, Tiêu hồng Hương giữa: Ho..."
3,1003772251,Dsquared2 Icon Pour Femme,Dsquared2 Icon Pour Femme - 100ml,dsquared2-icon-pour-femme,110100204697,2024,Ý,2200000,2211000,0,39,2,DSQUARED2,[Nữ],"[Tươi mát, Tinh tế, Trẻ trung]","[Hương Cam quýt, Hoa nhài, Kẹo praline]",0.0,0,0,0,0,0,0,"Hương đầu: Cam chanh, Tiêu hồng Hương giữa: Ho..."
4,1003771959,Nishane Hacivat,Nishane Hacivat - 100ml Tester,nishane-hacivat-1,110100204696,2017,Thổ Nhĩ Kỳ,7850000,0,1003771949,0,0,Nishane,[Unisex],"[Tinh tế, Trưởng Thành, Cuốn hút]","[Hương rêu sồi, Quả dứa, Bưởi]",0.0,0,0,0,0,0,0,"Hương đầu: Dứa, Bưởi, Cam bergamot Hương giữa:..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3295,10950126,Jo Malone London Mimosa & Cardamom,Jo Malone London Mimosa & Cardamom - 100ml,nuoc-hoa-unisex-jo-malone-mimosa-cardamom,110100200330,2015,Anh,2950000,3500000,0,0,0,Jo Malone,[Nữ],"[Thanh lịch, Sang trọng, Dịu dàng]","[Hoa mimosa, Bạch đậu khấu, Đậu tonka]",5.0,1,0,0,0,0,100,
3296,10950125,Jo Malone London Grapefruit,Jo Malone London Grapefruit - 30ml,nuoc-hoa-unisex-jo-malone-grapefruit,110100200328,1992,Anh,1950000,2400000,0,0,0,Jo Malone,[Unisex],"[Tươi mới, Sang trọng, Tinh tế]","[Quả bưởi, Hoa hương thảo, Cỏ hương bài, Bạc hà]",4.5,2,0,0,0,50,50,
3297,10950123,Jo Malone London Gardenia & Oud Absolu,Jo Malone London Gardenia & Oud Absolu - 100ml,jo-malone-london-gardenia-oud-absolu,110100202113,2020,Anh,4350000,4350000,0,0,0,Jo Malone,[Unisex],"[Tinh tế, Sang trọng, Thu Hút]","[Trầm hương, Hoa dành dành]",4.0,1,0,0,0,100,0,
3298,10950121,Jo Malone London French Lime Blossom,Jo Malone London French Lime Blossom - 30ml,nuoc-hoa-nu-jo-malone-french-lime-blossom,110100200326,1995,Anh,1950000,2400000,0,0,0,Jo Malone,[Uni],"[Tươi mát, Nhẹ nhàng, Gợi cảm]","[Sáp tổ ong, Cam bergamot, Hoa linh lan, Hoa q...",5.0,1,0,0,0,0,100,


In [ ]:
full_df.to_csv('../data/namperfume_product.csv', index=0)